In [ ]:
#pip install --upgrade pycaret
#pip install python-dotenv
#pip install SQLAlchemy
#pip install pymssql

#sudo openconnect --user="gmejia" --authgroup="VPN_EMPLEADOS" "vpn.unetrans.com.co"

In [18]:
from pycaret.time_series import TSForecastingExperiment

from sqlalchemy import create_engine


import pandas as pd
import numpy as np

import datetime
import random

import plotly.express as px

import sys
import os

from dotenv import load_dotenv

In [20]:
load_dotenv()

user=os.getenv('user')
password=os.getenv('password')
localhost=os.getenv('localhost')
port=os.getenv('port')
database=os.getenv('database')

engine=create_engine(f'mssql+pymssql://{user}:{password}@{localhost}:{port}/{database}', echo=False)
connection = engine.connect()

file_path=os.path.join('data','incidents.sql')
with open(file_path, 'r') as f:
    query = f.read()

df = pd.read_sql(query, connection)

connection.close()

df

ObjectNotExecutableError: Not an executable object: "SELECT * FROM historico_incidentes\nWHERE Fecha_Creacion >= '2023-01-01';"

In [ ]:
class Mefisto:
    def __init__(self, *args,**kwargs):
        
        self.create_terms()
        self.load_data()
        self.create_record()
        self.create_predict()
        self.create_data()
        self.create_file()

    def create_terms(self,*args,**kwargs):

        targets=[
            {"type_incident":"001 Accidente de Tr\u00e1nsito Solo Da\u00f1os (942)","type_finalized":"Negativo en sitio","state":1},
            {"type_incident":"001 Accidente de Tr\u00e1nsito Solo Da\u00f1os (942)","type_finalized":"Desis. en el sitio","state":1},
            {"type_incident":"001 Accidente de Tr\u00e1nsito Solo Da\u00f1os (942)","type_finalized":"Positivo","state":1},
            {"type_incident":"001 Accidente de Tr\u00e1nsito Solo Da\u00f1os (942)","type_finalized":"Falsa alarm en sitio","state":1},
            {"type_incident":"001 Accidente de Tr\u00e1nsito Solo Da\u00f1os (942)","type_finalized":"Fuera Mede. en sitio","state":1},
            {"type_incident":"002 Lesiones accidente de Tr\u00e1nsito (979)","type_finalized":"Positivo","state":1},
            {"type_incident":"002 Lesiones accidente de Tr\u00e1nsito (979)","type_finalized":"Negativo en sitio","state":1},
            {"type_incident":"002 Lesiones accidente de Tr\u00e1nsito (979)","type_finalized":"Fuera Mede. en sitio","state":1},
            {"type_incident":"002 Lesiones accidente de Tr\u00e1nsito (979)","type_finalized":"Desis. en el sitio","state":1},
            {"type_incident":"002 Lesiones accidente de Tr\u00e1nsito (979)","type_finalized":"Falsa alarm en sitio","state":1},
            {"type_incident":"003 Muerte Accidente de Tr\u00e1nsito (980A)","type_finalized":"Positivo","state":1},
            {"type_incident":"003 Muerte Accidente de Tr\u00e1nsito (980A)","type_finalized":"Desis. en el sitio","state":1},
            {"type_incident":"003 Muerte Accidente de Tr\u00e1nsito (980A)","type_finalized":"Falsa alarm en sitio","state":1},
            {"type_incident":"003 Muerte Accidente de Tr\u00e1nsito (980A)","type_finalized":"Negativo en sitio","state":1},
            {"type_incident":"003 Muerte Accidente de Tr\u00e1nsito (980A)","type_finalized":"Fuera Mede. en sitio","state":1},
            {"type_incident":"009 Servicio de Apoyo Tr\u00e1nsito (943)","type_finalized":"Positivo","state":1},
            {"type_incident":"009 Servicio de Apoyo Tr\u00e1nsito (943)","type_finalized":"Negativo en sitio","state":1},
            {"type_incident":"009 Servicio de Apoyo Tr\u00e1nsito (943)","type_finalized":"Desis. en el sitio","state":1},
            {"type_incident":"009 Servicio de Apoyo Tr\u00e1nsito (943)","type_finalized":"Fuera Mede. en sitio","state":1},
            {"type_incident":"009 Servicio de Apoyo Tr\u00e1nsito (943)","type_finalized":"Falsa alarm en sitio","state":1}
        ]

        turns={
            0:"3. noche",
            1:"3. noche",
            2:"3. noche",
            3:"3. noche",
            4:"3. noche",
            5:"1. manana",
            6:"1. manana",
            7:"1. manana",
            8:"1. manana",
            9:"1. manana",
            10:"1. manana",
            11:"1. manana",
            12:"1. manana",
            13:"2. tarde",
            14:"2. tarde",
            15:"2. tarde",
            16:"2. tarde",
            17:"2. tarde",
            18:"2. tarde",
            19:"2. tarde",
            20:"2. tarde",
            21:"3. noche",
            22:"3. noche",
            23:"3. noche",
        }

        features={
            'Incidente':'id_incident',
            'Fecha_Creacion':'date_time_created',
            'Fecha_Finalizacion':'date_time_finalized',
            'Tipo_Incidente':'type_incident',
            'Tipo_Finalizacion':'type_finalized',
            'Latitud':'latitude',
            'Longitud':'longitude',
        }


        self.features=features
        self.turns=turns
        self.targets=targets


    def load_data(self,*args,**kwargs):
        file_path = os.path.join('data','data.csv')
        df=pd.read_csv(file_path, low_memory=False)
        self.df_base=df

    def create_record(self,*args,**kwargs):
        
        features=self.features.copy()
        turns=self.turns.copy()
        targets=self.targets.copy()

        df=self.df_base.copy()

        df.rename(columns=features,inplace=True)

        df_tmp=pd.DataFrame(targets)
        df= pd.merge(df_tmp,df)

        df['date_time_created']=pd.to_datetime(df['date_time_created'])
        df['date_time_finalized']=pd.to_datetime(df['date_time_finalized'])

        date_horizon=datetime.datetime(2023,1,1).strftime(format=r'%Y-%m-%d')
        df.query(f"date_time_created >='{date_horizon}'", engine='python',inplace=True)

        df['date_time_created'] = df['date_time_created'].dt.round('1h')
        df['date_time_finalized'] = df['date_time_finalized'].dt.round('1h')     

        df['time'] = df.apply(lambda x: pd.date_range(start=x['date_time_created'], end=x['date_time_finalized'], freq='H').to_list(), axis=1)

        df=df.explode(column=['time'])

        groups = [
            pd.Grouper(key="time", freq="H")
        ]

        aggs = {
            "id_incident": ["nunique"]
        }

        df = df.groupby(groups).agg(aggs).reset_index()
        df.columns = df.columns.map("_".join).str.strip("_")


        df['id_incident_nunique']=df['id_incident_nunique'].rolling(8).mean().round(0)

        df.dropna(inplace=True)
        self.df_record=df

    def create_predict(self,*args,**kwargs):
        #estimator="huber_cds_dt"
        estimator="dt_cds_dt"

        fh=24*30*6
        verbose=True

        df=self.df_record.copy()
        
        forecast = TSForecastingExperiment()

        columns=['time','id_incident_nunique']
        forecast.setup(data=df[columns].set_index(keys='time',inplace=False),verbose=verbose, session_id=123, use_gpu=True,fold=5)


        forecast_mdl=forecast.create_model(**dict(estimator=f"{estimator}", verbose=verbose))
        forecast_mdl=forecast.tune_model(**dict(estimator=forecast_mdl, verbose=verbose))
        forecast_mdl=forecast.finalize_model(**dict(estimator=forecast_mdl))

        forecast.plot_model(estimator=forecast_mdl, data_kwargs = {'fh' : fh})

        df_tmp=forecast.predict_model(**dict(estimator=forecast_mdl, return_pred_int=False, fh=fh, round=0, verbose=verbose))
        
        df_tmp.rename(columns=dict(y_pred="id_incident_nunique"),inplace=True)
        
        df_tmp["time"]=df_tmp.index.to_timestamp()
        df_tmp["time"]=pd.to_datetime(df_tmp["time"])

        df_tmp.reset_index(drop=True,inplace=True)
        
        self.df_forecast=df_tmp

    def create_data(self,*args,**kwargs):
        turn=self.turns.copy()
        
        df=self.df_record.copy()
        df['state']='real'

        df_tmp=self.df_forecast.copy()
        df_tmp['state']='forecast'

        df=pd.concat([df,df_tmp],ignore_index=True)
        
        df['year'] = df['time'].dt.year
        df['month'] = df['time'].dt.month
        df['day'] = df['time'].dt.day

        df['day_of_week'] = df['time'].dt.day_of_week
        df['week'] = df['time'].dt.week

        df['date'] = df['time'].dt.date
        df['hour'] = df['time'].dt.hour

        df['turn'] = df['hour'].map(turn)



        df['resource_max']=df['id_incident_nunique'].rolling(8).max().round(0)
        df['resource_mean']=df['id_incident_nunique'].rolling(8).mean().round(0)
        df['resource_min']=df['id_incident_nunique'].rolling(8).min().round(0)

        df.dropna(inplace=True)

        df['id_incident_nunique'] = df['id_incident_nunique'].round(0).astype(int)

        df['resource_max'] = df['resource_max'].round(0).astype(int)
        df['resource_mean'] = df['resource_mean'].round(0).astype(int)
        df['resource_min'] = df['resource_min'].round(0).astype(int)

        fig=px.line(df,x='time',y=['id_incident_nunique','resource_max','resource_mean','resource_min'])

        self.fig = fig

        self.df_data=df
    def create_file(self,*args,**kwargs):
        df=self.df_data.copy()
        fig=self.fig
        
        file_path = os.path.join('data','rst.csv')
        df.to_csv(file_path,index=False)

        file_path = os.path.join('data','rst.html')
        fig.write_html(file_path)
        fig.show()

In [ ]:
mefisto=Mefisto()

In [ ]:
df=mefisto.df_data
df
